# 微调模型评估
除了上述列举的各种任务类型，还可以通过设计提示词让大模型充当“裁判”的角色对已经微调好的模型进行打分，比如让微调后的模型回答一些问题，如果是数学问题可以直接对答案，而如果是一些文本类的回答，则可以让大模型对回答的准确性进行打分，充当“裁判”的大模型要求能力很强，这样才能在评估过程中客观评分，我们举一个简单的例子：

In [1]:
### 加载模型Qwen模型
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name_or_path_instruct = '/home/lixinyu/weights/Qwen2.5-3B-Instruct'  # 替换为你下载的模型路径
tokenizer_instruct = AutoTokenizer.from_pretrained(model_name_or_path_instruct)
model_instruct = AutoModelForCausalLM.from_pretrained(model_name_or_path_instruct,device_map='cuda:0', torch_dtype='auto')

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.71it/s]


In [2]:
# 推理函数
def inference(model,tokenizer,prompt,system="你是一个专业的人工智能助手",max_new_tokens=512,temperature=0.9):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
    )
    # 提取仅由模型生成的token ids（排除输入部分）
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    # 统计生成的tokens数量（直接取生成序列的长度）
    generated_tokens_count = len(generated_ids[0])  # 因输入是单条，取第一个元素的长度
    # 解码生成的token ids为文本
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 返回响应文本和生成的tokens数量
    return {
        "response":response,
        "generated_tokens_count":generated_tokens_count
    }

In [3]:
prompt_1="""
你是一位严格的评分老师。下面是一段「待评回答」。请只看它的事实准确性，忽略格式和语气。  
评分标准：  
0 分 = 完全不符合事实
1 分 = 有明显事实错误  
2 分 = 有小错误，整体仍对  
3 分 = 完全正确  

只要回复一个数字（0/1/2/3）和一句理由，不要多余内容。

### 待评估问题：
巴黎现在有多少的人口？

### 待评回答：
根据2022年最新的统计数据，巴黎的总人口约为216万。
"""

In [4]:
print(inference(model_instruct,tokenizer_instruct,prompt_1))


{'response': '2/巴黎2022年的最新统计数据并没有明确指出人口为216万，存在一定的不确定性。', 'generated_tokens_count': 25}


当然我们也可以让大模型判断回答结果和正确结果之间的相似程度作为评分标准：

In [5]:
prompt_2="""
你是评分员。请把下面的“标准答案”和“模型回答”对比，按 1–5 打分：  
5 分 = 含义完全一致  
4 分 = 基本一致，细节微差  
3 分 = 部分匹配  
2 分 = 少量相关  
1 分 = 完全不符  

只回复一个数字（1-5）和一句理由，不要多余内容。

### 问题：
光合作用需要什么？

### 标准答案：
阳光  

### 模型回答：
光合作用是植物、藻类和某些细菌利用阳光将二氧化碳和水转化为氧气和葡萄糖的过程。在这个过程中，光合作用需要以下条件和成分：

1. **光**：这是光合作用的直接能量来源。
2. **二氧化碳（CO₂）**：从大气中吸收的二氧化碳用于合成有机物。
3. **水（H₂O）**：通过叶绿体中的叶绿体膜上的水裂解过程产生氢离子和氧气。
4. **叶绿素和其他色素**：叶绿素是主要的色素，能够吸收太阳光并将其转化为
"""

In [6]:
print(inference(model_instruct,tokenizer_instruct,prompt_2))

{'response': '4\n\n理由：模型回答虽然提到了光作为光合作用的能量来源，但没有直接给出“阳光”这一术语，只是表述为“光”。其他细节与标准答案基本一致。', 'generated_tokens_count': 42}


从上面的例子可以看出，当大模型充当“裁判”后就可以替代人工做出合理的打分，这样的评估可以通过批量运行脚本，从而提高评估的效率。